In [1]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv(
    "/Users/rahulanil/garchomp/projects/kaggle/titanic/data/train.csv"
)
test_df = pd.read_csv("/Users/rahulanil/garchomp/projects/kaggle/titanic/data/test.csv")


In [3]:
import titanic_preprocessing as tp

# generic_perprocessing
test_df_passengeId = test_df["PassengerId"]
train_df, test_df = tp.generic_perprocessing(train_df, test_df)

In [4]:
X = train_df.loc[:, train_df.columns != "Survived"]
y = train_df["Survived"]

In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(
    [("onehot", OneHotEncoder(sparse=False), ["Pclass", "Sex", "Embarked", "Initials"])]
)

ct.fit(X)
X_ct = ct.transform(X)
print(f"X_ct shape: {X_ct.shape}")

test = ct.transform(test_df)
print(f"X_test shape: {test.shape}")

X_ct shape: (889, 16)
X_test shape: (418, 16)


In [6]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier   

param_grid = {
    "eta": [i for i in np.arange(0, 1, 0.1)],
    "max_depth": [i for i in range(2, 10)],
    "n_estimators": [i for i in range(10, 200, 10)],
}

grid_search = GridSearchCV(
    XGBClassifier(),
    param_grid,
    cv=5,
    return_train_score=True,
    n_jobs=-1,
    verbose=10,
)



In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_ct, y, random_state=0)

grid_search.fit(X_train, y_train)
print(f"test scores: {grid_search.score(X_test, y_test)}")
print(f"Best parametesrs: {grid_search.best_params_}")
print(f"best cross validation score: {grid_search.best_score_}")


Fitting 5 folds for each of 1520 candidates, totalling 7600 fits
[CV 3/5; 1/1520] START eta=0.0, max_depth=2, n_estimators=10....................
[CV 4/5; 1/1520] START eta=0.0, max_depth=2, n_estimators=10....................
[CV 1/5; 2/1520] START eta=0.0, max_depth=2, n_estimators=20....................
[CV 2/5; 1/1520] START eta=0.0, max_depth=2, n_estimators=10....................
[CV 1/5; 1/1520] START eta=0.0, max_depth=2, n_estimators=10....................
[CV 5/5; 1/1520] START eta=0.0, max_depth=2, n_estimators=10....................
[CV 2/5; 2/1520] START eta=0.0, max_depth=2, n_estimators=20....................
[CV 3/5; 1/1520] END eta=0.0, max_depth=2, n_estimators=10;, score=(train=0.627, test=0.624) total time=   0.0s
[CV 4/5; 1/1520] END eta=0.0, max_depth=2, n_estimators=10;, score=(train=0.627, test=0.624) total time=   0.0s
[CV 1/5; 1/1520] END eta=0.0, max_depth=2, n_estimators=10;, score=(train=0.626, test=0.627) total time=   0.0s
[CV 2/5; 1/1520] END eta=0.0, ma

In [8]:
grid_search_results = pd.DataFrame(grid_search.cv_results_)

In [9]:
best_params = grid_search.best_params_
print(best_params)
clf = XGBClassifier(**best_params)
clf.fit(X_ct, y)
y_test = pd.Series(clf.predict(test), name="Survived").astype(int)
result = pd.concat([test_df_passengeId, y_test], axis=1)
result.to_csv("XGB.csv", index=False)
display(result)

{'eta': 0.2, 'max_depth': 3, 'n_estimators': 20}


NameError: name 'GradientBoostingClassifier' is not defined